In [34]:
import pandas as pd
import string
import re

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
# from tqdm import tqdm
# from tensorflow.keras.preprocessing.text import one_hot

In [23]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each doc.
MAX_SEQUENCE_LENGTH = 5489
# This is fixed.
EMBEDDING_DIM = 100

In [3]:
df = pd.read_csv(r'./data/parsed_data.csv')

In [20]:
f = [len(x.split()) for x in df['body']]

In [22]:
max(f)

5489

In [4]:
df['label'] = df['label'].apply(lambda x: x.lower())

true_labels = ['correct attribution','mostly true','mostly-true', 'true',]
false_labels = ['false','mostly false','pants-fire',]
temp=[]
for index in range(len(df)):
    if df['label'][index] in true_labels:
        temp.append(0)
    elif df['label'][index] in false_labels:
        temp.append(1)
    else:
        temp.append(2)
        
df['new_label']=temp

In [31]:
Y = pd.get_dummies(df['new_label']).values

In [24]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['body'].values)
X = tokenizer.texts_to_sequences(df['body'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 86738 unique tokens.


In [27]:
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (22298, 5489)


In [35]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(20068, 5489) (20068, 3)
(2230, 5489) (2230, 3)


In [11]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

import numpy as np

In [13]:
embedding_vector_fetures = 100
vocab_size = 5000

model = Sequential()
model.add(Embedding(vocab_size,embedding_vector_fetures,input_length= MAX_SEQUENCE_LENGTH))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))#LSTM(100))
model.add(Dense(3, activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 64)           320000    
                                                                 
 lstm (LSTM)                 (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 24)                1560      
                                                                 
Total params: 354,584
Trainable params: 354,584
Non-trainable params: 0
_________________________________________________________________


In [15]:
final_x = np.array(embedded_docs)
final_y = np.array(td['label'])

In [17]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(final_x, final_y, test_size = 0.2, random_state = 42)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)


(17838, 300) (4460, 300) (17838,) (4460,)


In [20]:
import tensorflow as tf
final_y_train = tf.keras.utils.to_categorical(y_train, num_classes=24)
final_y_test = tf.keras.utils.to_categorical(y_test, num_classes=24)

In [21]:
print(final_y_train.shape, final_y_test.shape)

(17838, 24) (4460, 24)


In [22]:
model.fit(x_train, final_y_train, batch_size=32, epochs=10, validation_data=(x_test,final_y_test))

Epoch 1/10


2023-04-08 19:45:22.609345: I tensorflow/core/common_runtime/executor.cc:1197] [/job:localhost/replica:0/task:0/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: indices[20,205] = 7518 is not in [0, 5000)
	 [[{{node sequential/embedding/embedding_lookup}}]]


InvalidArgumentError: Graph execution error:

Detected at node 'sequential/embedding/embedding_lookup' defined at (most recent call last):
    File "/home/ubuntu/miniconda3/envs/tf/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/ubuntu/miniconda3/envs/tf/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/ubuntu/miniconda3/envs/tf/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/ubuntu/miniconda3/envs/tf/lib/python3.9/site-packages/traitlets/config/application.py", line 992, in launch_instance
      app.start()
    File "/home/ubuntu/miniconda3/envs/tf/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 711, in start
      self.io_loop.start()
    File "/home/ubuntu/miniconda3/envs/tf/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever

In [23]:
model.fit(x_train, y_train, validation_data = (x_test,y_test), epochs = 10,batch_size = 64)

Epoch 1/10


2023-04-08 19:45:40.715815: I tensorflow/core/common_runtime/executor.cc:1197] [/job:localhost/replica:0/task:0/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: indices[37,101] = 6656 is not in [0, 5000)
	 [[{{node sequential/embedding/embedding_lookup}}]]


InvalidArgumentError: Graph execution error:

Detected at node 'sequential/embedding/embedding_lookup' defined at (most recent call last):
    File "/home/ubuntu/miniconda3/envs/tf/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/ubuntu/miniconda3/envs/tf/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/ubuntu/miniconda3/envs/tf/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/ubuntu/miniconda3/envs/tf/lib/python3.9/site-packages/traitlets/config/application.py", line 992, in launch_instance
      app.start()
    File "/home/ubuntu/miniconda3/envs/tf/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 711, in start
      self.io_loop.start()
    File "/home/ubuntu/miniconda3/envs/tf/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever